In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Source
from IPython.display import display, SVG
from scipy.sparse import load_npz
import tensorflow as tf

Įkeliu failą flatland_train.npz iš savo kompiuterio:

In [4]:
from google.colab import files


uploaded = files.upload()

Saving flatland_train.npz to flatland_train.npz


In [5]:
import io

Pasikraunu duomenis:

In [6]:
data = np.load(io.BytesIO(uploaded['flatland_train.npz']))

In [7]:
data

In [8]:
print(data.files)

['X', 'y', 'allow_pickle']


Paruošiu duomenis:

In [9]:
X = data['X']
y = data['y']
y[y != 0] -= 2    # Correct labels so that triangle is mapped to class 1
X = X / 255.      # Scale down to range [0, 1]

Atskiriame testinius duomenis:

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3, random_state=42)

In [11]:
print(X_train.shape)
print(y_train.shape)

(7000, 50, 50)
(7000,)


Pridedam triukšmo (nesėkmingas bandymas)



In [ ]:
#noise_stddev = 0.1
#noise = tf.random.normal(shape=tf.shape(X_train), mean=0.0, stddev=noise_stddev, dtype=tf.float32)
#noisy_data = X_train + noise

In [12]:
from collections import Counter
Counter(y_train[:])

Counter({1.0: 2005, 0.0: 925, 3.0: 1508, 4.0: 1243, 2.0: 1319})

In [13]:
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy

Modeliai eina prastėjimo tvarka

Pats geriausias modelis:

In [14]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size = (3,3), activation="relu", kernel_initializer = 'he_uniform', padding = 'same', input_shape=[50,50,1]))
model.add(BatchNormalization())
model.add(keras.layers.Conv2D(32, kernel_size = (3,3), activation="relu", kernel_initializer = 'he_uniform', padding = 'same'))
model.add(BatchNormalization())
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Conv2D(64, kernel_size = (3,3), activation="relu", kernel_initializer = 'he_uniform', padding = 'same'))
model.add(BatchNormalization())
model.add(keras.layers.Conv2D(64, kernel_size = (3,3), activation="relu", kernel_initializer = 'he_uniform', padding = 'same'))
model.add(BatchNormalization())
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Conv2D(128, kernel_size = (3,3), activation="relu", kernel_initializer = 'he_uniform', padding = 'same'))
model.add(BatchNormalization())
model.add(keras.layers.Conv2D(128, kernel_size = (3,3), activation="relu", kernel_initializer = 'he_uniform', padding = 'same'))
model.add(BatchNormalization())
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Dropout(0.4))
#model.add(keras.layers.Conv2D(10, kernel_size = (3,3)))
#model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation="relu",kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(keras.layers.Dropout(0.5))
#model.add(keras.layers.Dense(256, activation="relu"))
#model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
               metrics=[SparseCategoricalAccuracy()])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 50, 50, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 50, 50, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 50, 50, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 25, 25, 32)        0         
 D)                                                              
                                                        

Prastesnis modelis:

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(10, kernel_size = (3,3), input_shape=[50,50,1]))
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Conv2D(10, kernel_size = (3,3)))
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
               metrics=[SparseCategoricalAccuracy()])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 46, 46, 10)        260       
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 23, 23, 10)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 19, 19, 10)        2510      
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 9, 9, 10)          0         
 ng2D)                                                           
                                                                 
 flatten_5 (Flatten)         (None, 810)               0         
                                                                 
 dense_35 (Dense)            (None, 256)             

Dar prastesnis modelis:

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[50,50]))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
               metrics=[SparseCategoricalAccuracy()])
model.summary()

Pats prasčiausias modelis:

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[50, 50]))
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="categorical_crossentropy",
               optimizer=keras.optimizers.Adam(learning_rate=3e-4),
             metrics=[SparseCategoricalAccuracy()])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 2500)              0         
                                                                 
 dense_4 (Dense)             (None, 64)                160064    
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 10)                330       
                                                                 
Total params: 162474 (634.66 KB)
Trainable params: 162474 (634.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Nesėkmingas bandymas su mixup

In [ ]:
# def mixup(images, labels, alpha=0.2):
#     batch_size = tf.shape(images)[0]
#     lambda_value = tf.random.uniform(shape=[batch_size, 1, 1, 1], minval=0.0, maxval=alpha)

#     # Shuffle the indices to pair images and labels for mixing
#     shuffle_indices = tf.random.shuffle(tf.range(batch_size))
#     mixed_images = lambda_value * images + (1 - lambda_value) * tf.gather(images, shuffle_indices)
#     mixed_labels = lambda_value * labels + (1 - lambda_value) * tf.gather(labels, shuffle_indices)

#     return mixed_images, mixed_labels

# alpha = 0.2
# mixed_images, mixed_labels = mixup(X_train, y_train, alpha)



In [ ]:
#loss = model.fit(mixed_images, mixed_labels, epochs=70, batch_size = 256, validation_data = (X_test, y_test))

In [ ]:
from keras.callbacks import EarlyStopping

Treniruojame modelį:

In [15]:
loss = model.fit(X_train, y_train, epochs=70, batch_size = 256, validation_data = (X_test, y_test))

Epoch 1/70
28/28 [==============================] - 20s 152ms/step - loss: 2.6254 - sparse_categorical_accuracy: 0.2844 - val_loss: 1.8840 - val_sparse_categorical_accuracy: 0.3667
Epoch 2/70
28/28 [==============================] - 3s 99ms/step - loss: 1.9219 - sparse_categorical_accuracy: 0.4071 - val_loss: 1.9616 - val_sparse_categorical_accuracy: 0.2917
Epoch 3/70
28/28 [==============================] - 3s 95ms/step - loss: 1.4414 - sparse_categorical_accuracy: 0.5450 - val_loss: 2.9728 - val_sparse_categorical_accuracy: 0.1830
Epoch 4/70
28/28 [==============================] - 3s 95ms/step - loss: 1.0726 - sparse_categorical_accuracy: 0.6526 - val_loss: 5.2419 - val_sparse_categorical_accuracy: 0.1830
Epoch 5/70
28/28 [==============================] - 3s 101ms/step - loss: 0.8114 - sparse_categorical_accuracy: 0.7440 - val_loss: 7.3457 - val_sparse_categorical_accuracy: 0.1830
Epoch 6/70
28/28 [==============================] - 3s 104ms/step - loss: 0.6202 - sparse_categorical_

Pasižiūrime modelio tikslumą:

In [16]:
pred = model.predict(X_test).argmax(axis=1)
print('Accuracy on test set - {0:.02%}'.format((pred == y_test).mean()))

94/94 [==============================] - 1s 5ms/step
Accuracy on test set - 98.23%


Išsaugojame modelį:

In [ ]:
# Save the model and upload it to git
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
